# KerasNLP ile sıfırdan GPT metni oluşturma

Metin üretimi için bir mini GPT modeli eğitmek üzere KerasNLP'yi kullanacağız.

## Giriş

Bu örnekte, küçük bir generative oluşturmak için KerasNLP'yi kullanacağız.
GPT, girilen metin bilgisinden sonra gelişmiş metin oluşturmanıza izin veren Transformer tabanlı bir modeldir.

Modelimizi [simplebooks-92](https://arxiv.org/abs/1911.12391) corpusu üzerinde eğiteceğiz. Bir kaç farklı romandan oluşan bir veri kümesidir.

Bu örnek,
[Minyatür GPT ile metin oluşturma](https://keras.io/examples/generative/text_generation_with_miniature_gpt/)
KerasNLP soyutlamaları kavramlarını birleştirir. KerasNLP tokenizasyonunun, katmanlarının ve
ölçümler eğitimi basitleştirir.

Bu örneği google colab üzerinde çalıştırıyorsan çalışma türünü GPU olarak ayarlamalısın.

Sonra ie KerasNLP yi indirelim.
`pip install keras-nlp`

`KerasNLP`, kullanıcıları tüm geliştirme döngüleri boyunca destekleyen doğal bir dil işleme kitaplığıdır. İş akışlarımız, kutudan çıkar çıkmaz kullanıldığında son teknoloji önceden ayarlanmış ağırlıklara ve mimarilere sahip olan ve daha fazla kontrol gerektiğinde kolayca özelleştirilebilen modüler bileşenlerden oluşturulmuştur. Geliştiricilerin TensorFlow ekosistemini kullanarak kolay üretim bekleyebilmesi için tüm iş akışları için grafik içi hesaplamayı vurguluyoruz.

Bu kitaplık, çekirdek Keras API'sinin bir uzantısıdır.

In [1]:
!pip install --upgrade protobuf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras-nlp

## Gerekli paketlerimizi import edelim

In [3]:
import os
import keras_nlp
import tensorflow as tf
from tensorflow import keras

## Ayarlar ve hiperparametreler

In [4]:
# Data
BATCH_SIZE = 64
SEQ_LEN = 128
MIN_TRAINING_SEQ_LEN = 450

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 5000  #Modeldeki parametreleri sınırlar.

# Eğitim
EPOCHS = 6

# Çıkarım
NUM_TOKENS_TO_GENERATE = 80

## Data'yı yükleyelim.

Şimdi veri setini indirelim. SimpleBooks veri seti 1.573 Gutenberg kitabından oluşur ve
kelime düzeyinde belirteç oranına en küçük kelime boyutundan biri, ~98k kelime dağarcığı boyutuna sahiptir.

In [5]:
keras.utils.get_file(
    origin="https://dldata-public.s3.us-east-2.amazonaws.com/simplebooks.zip",
    extract=True,
)
dir = os.path.expanduser("~/.keras/datasets/simplebooks/")

# simplebooks-92 train setini yükleyin ve kısa satırları filtreleyin.
raw_train_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/train.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

# simplebooks-92 doğrulama setini yükleyin ve kısa satırları filtreleyin.
raw_val_ds = (
    tf.data.TextLineDataset(dir + "simplebooks-92-raw/valid.txt")
    .filter(lambda x: tf.strings.length(x) > MIN_TRAINING_SEQ_LEN)
    .batch(BATCH_SIZE)
)

282386239/282386239 [==============================] - 10s 0us/step


## Tokenizer'ı eğitelim

Tokenizerı, "VOCAB_SIZE" kelime dağarcığı boyutu için eğitim veri kümesinden eğitiyoruz,
"VOCAB_SIZE" yukarıda ayarlamış olduğumuz bir hiperparametredir. Kelime dağarcığını mümkün olduğunca sınırlamak istiyoruz, çünkü model parametre sayısı üzerinde büyük bir etkisi vardır.

- `"[PAD]"` kısa cümlelerin sonuna ya da başına varsayılan olarak 0 rakamı eklenir ki modele girecek olan cümlenin boyutunu aynı tutalım.

- `"[UNK]"` varsayılanla eşleşmesi gereken OOV alt sözcükleri için `oov_token="[UNK]"`

- `"[BOS]"` cümlenin başlangıcı anlamına gelir,
eğitim verilerinin her satırının başlangıcını temsil eder.

In [6]:
# Train tokenizer vocabulary
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

## Tokenizer'ı yükleyelim

`keras_nlp.tokenizers.WordPieceTokenizer` kullanarak WordPieceTokenizer'ı başlatabiliriz. Bu paket
BERT ve diğer modeller tarafından kullanılan WordPiece algoritmasının uygulanmasıdır. Verimli şekilde metin ön işleme yapabiliriz.


In [7]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

## Tokenize Verisini ayarlıyoruz. ( özellik ve etiketi şeklinde )

Veri kümesini belirteç haline getirerek ve onu `features` ve `labels` olarak bölerek önceden işleriz.

In [8]:
# paketleyici bir başlangıç belirteci ekler
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)


def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels


# Tokenize edin ve train ve label dizilerine bölün.
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

## Modelimizin İnşaası

Aşağıdaki katmanlarla küçültülmüş GPT modelimizi oluşturuyoruz:

- Token ve konumu için gömme matrisini içinde tutan bir adet `keras_nlp.layers.TokenAndPositionEmbedding` katmanı.
- Varsayılan nedensel maskeleme ile birden çok `keras_nlp.layers.TransformerDecoder` katmanı.
Katman, yalnızca kod çözücü dizisiyle çalıştırıldığında çapraz dikkat göstermez.
- Son bir yoğun doğrusal katman

In [9]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)
# Transformer decoders.
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)  # Tek bir argüman vermek, yalnızca çapraz ilgiyi atlar.
# Çıkış.
outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=loss_fn, metrics=[perplexity])

Model özetimize bir göz atalım. 
Parametrelerin büyük çoğunluğu token_and_position_embedding ve çıktı katmanındadır! Bu, kelime boyutunun (VOCAB_SIZE) modelin boyutu üzerinde büyük bir etkiye sahip olduğu, ancak Transformer kod çözücü katmanlarının (NUM_LAYERS) sayısının onu çok fazla etkilemediği anlamına gelir.

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        1312768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 5000)        128500

## Eğitim

Artık modelimize sahip olduğumuza göre, onu fit() metodu ile eğitelim.

In [11]:
model.fit(train_ds, validation_data=val_ds, verbose=2, epochs=EPOCHS)

Epoch 1/6
3169/3169 - 403s - loss: 4.5896 - perplexity: 98.8414 - val_loss: 4.1571 - val_perplexity: 64.4688 - 403s/epoch - 127ms/step
Epoch 2/6
3169/3169 - 250s - loss: 4.0607 - perplexity: 58.2387 - val_loss: 3.9988 - val_perplexity: 55.0357 - 250s/epoch - 79ms/step
Epoch 3/6
3169/3169 - 248s - loss: 3.9406 - perplexity: 51.6441 - val_loss: 3.9409 - val_perplexity: 51.9935 - 248s/epoch - 78ms/step
Epoch 4/6
3169/3169 - 244s - loss: 3.8762 - perplexity: 48.4238 - val_loss: 3.8679 - val_perplexity: 48.3114 - 244s/epoch - 77ms/step
Epoch 5/6
3169/3169 - 239s - loss: 3.8314 - perplexity: 46.2981 - val_loss: 3.8806 - val_perplexity: 48.8037 - 239s/epoch - 76ms/step
Epoch 6/6
3169/3169 - 240s - loss: 3.7978 - perplexity: 44.7659 - val_loss: 3.8226 - val_perplexity: 46.1492 - 240s/epoch - 76ms/step


## Çıkarım

Eğitimli modelimizin performansını ölçmek için test edebiliriz. Çünkü bu model
`"[BOS]"` belirteci ile oluşturulmuş, metin oluşturma için boş bir başlangıç istemimiz olabilir. Bu durumu istemeyiz. Kontrol edelim.

In [12]:
# Doldurulmamış bos belirteci.
prompt_tokens = tf.convert_to_tensor([tokenizer.token_to_id("[BOS]")])

Çıkarım için `keras_nlp.utils` modülünü kullanacağız. Her metin üretimi model etrafında bir "token_logits_fn()" sarmalayıcı gerektirir. Bu sarmalayıcı dolgusuz bir belirteç dizisinde ve çıktı olarak bir sonraki belirtecin logitlerini gerektirir.

In [13]:

def token_logits_fn(inputs):
    cur_len = inputs.shape[1]
    output = model(inputs)
    return output[:, cur_len - 1, :]  # sonraki tokenleri döndür.


Sarmalayıcı işlevini oluşturmak, bu işlevleri kullanmanın en karmaşık kısmıdır. Artık bittiğine göre, açgözlü aramayla başlayarak farklı araçları test edelim.

## Üretilen çıktıları hangi mantıkla seçeceğiz, farklı arama ve seçim yöntemleri vardır. Bir kısmına göz atalım.

### Greedy search

Her zaman adımında en olası tokenleri açgözlülükle seçiyoruz. Başka bir deyişle, model çıktısının argmax'ını elde ederiz.

In [14]:
output_tokens = keras_nlp.utils.greedy_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Açgözlü arama oluşturulan metin: \n{txt}\n")

Açgözlü arama oluşturulan metin: 
b'[BOS] " i \' m not going to be a good man , " he said . " i \' m not going to be a good man , and i \' m not going to be a man , and i \' m going to be a man , and i \' m going to be a man , and i \' m going to be a man , and i \' m going to be a man ,'



Gördüğünüz gibi, açgözlü arama bir anlam ifade etmeye başlar, ancak hızla kendini tekrar etmeye başlar.
Bu, bazı yazılımlar tarafından düzeltilebilen metin oluşturma ile ilgili yaygın bir sorundur.
Olasılıksal metin oluşturma yardımcı programları daha sonra gösterilecektir!

### Beam search

Yüksek düzeyde ışın araması en olası dizilerin "num_beams" izini tutar.
Her zaman adımı ve tüm dizilerden bir sonraki en iyi tokeni tahmin eder.

Aç gözlü aramaya göre daha gelişmiştir ancak birden fazla olasılığı hesaplayıp tuttuğu için bellek ve hız açısından dejavantaj sunar.

**Not:** `num_beams=1` ayarını beam_search de ayarlarsak, açgözlü aramayla aynı işi yapmış oluruz.

In [15]:
output_tokens = keras_nlp.utils.beam_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    num_beams=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Beam search ile oluşturulan metin: \n{txt}\n")

Beam search ile oluşturulan metin: 
b'[BOS] " i don \' t know , " he said . " i don \' t know what to do , " he said . " i don \' t know what to do , but i don \' t know what to do . i don \' t know what to do . i don \' t know what to do , but i don \' t know what to do , and i don \' t know'



Aç gözlü arama gibi ışın araması hızla kendini tekrar etmeye başlar, çünkü hala
deterministik bir yöntem.
`Deterministik yöntem demek` sistemin gelecekteki durumlarının gelişmesinde rastgelelik bulunmayan bir sistem demektir.

### Random search

Rastgele arama, ilk olasılıksal yöntemimizdir. Her zaman adımında, bir sonrakini örnekler
model tarafından sağlanan softmax olasılıklarını kullanarak belirlenir.

In [16]:
output_tokens = keras_nlp.utils.random_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Random search ile üretilen metin: \n{txt}\n")

Random search ile üretilen metin: 
b'[BOS] edgar collector reperfusive the twosters number two of his best friends to close friends . a second lieutenant was taken suddenly an immense number of doubt . the distance was small , and grace had arranged it must be very pledge that she might for some purpose ; at any rate she would startle forward at once , because it would cost much if possible position ,'



Vay!, tekrar yok! Ancak, rastgele arama ile, bu örnekleme yöntemiyle kelime dağarcığındaki herhangi bir kelimenin çıkma şansı olduğundan, bazı anlamsız kelimelerin ortaya çıktığını görebiliriz. Bu, bir sonraki arama yardımcı programımız olan top-k search ile düzeltilebilir.

### Top-K search

Rastgele aramaya benzer şekilde, model tarafından sağlanan olasılık dağılımından bir sonraki tokenleri tahminliyoruz. Tek fark, burada en olası k belirteci seçip örneklemeden önce olasılık kütlesini bunların üzerine dağıtmamızdır. Bu şekilde, düşük olasılıklı belirteçlerden örnekleme yapmayacağız ve bu nedenle daha az saçma kelimemiz olacak :)

In [17]:
output_tokens = keras_nlp.utils.top_k_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    k=10,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-K search ile üretilen metin: \n{txt}\n")

Top-K search ile üretilen metin: 
b'[BOS] " the first thing is to go on , " she remarked , and her voice was peering through the glass . " when the girls came down to her house , she told her of a great crowd of men , but she had not been away to a moment , and had she been so long approved . she had heard nothing to do , and she had never heard of such a word'



### Top-P search

En iyi aramada bile geliştirilecek bir şeyler var. Top-k arama ile k sayısı sabittir, yani herhangi bir olasılık dağılımı için aynı sayıda belirteci seçer. Biri olasılık kütlesinin 2 kelime üzerinde yoğunlaştığı, diğeri ise olasılık kütlesinin eşit olarak 10 kelime üzerinde yoğunlaştığı iki senaryo düşünün. k=2'yi mi yoksa k=10'u mu seçmeliyiz? Burada her ikisine uyan bir değer yok. Ne yapmalıyız ?

Üst düzey aramanın devreye girdiği yer burasıdır! Bir k seçmek yerine, en iyi belirteçlerin olasılıklarının toplanmasını istediğimiz bir p olasılığı seçiyoruz. Bu şekilde, k'yi olasılık dağılımına göre dinamik olarak ayarlayabiliriz. p=0.9'u ayarlayarak, olasılık kütlesinin %90'ı ilk 2 belirteç üzerinde yoğunlaşırsa, örnek alınacak ilk 2 belirteci filtreleyebiliriz. Bunun yerine %90, 10 jetona dağıtılırsa, benzer şekilde örnek alınacak ilk 10 jetonu filtreleyecektir.

In [18]:
output_tokens = keras_nlp.utils.top_p_search(
    token_logits_fn,
    prompt_tokens,
    max_length=NUM_TOKENS_TO_GENERATE,
    p=0.5,
    from_logits=True,
)
txt = tokenizer.detokenize(output_tokens)
print(f"Top-P search ile üretilen metin: \n{txt}\n")

Top-P search ile üretilen metin: 
b'[BOS] " why should we go with you ? " he said . " i am very much afraid of being confirmitted , and if you are to be taken out of the city , it is impossible to take you into the town . it would be very hard to find fault , and would have made you feel so ashamed , that you would have been much better than you would have been'



### Metin oluşturmak için callbacks'leri kullanma

Yardımcı programları, bir tahminin çıktısını almanıza izin veren bir geri aramaya da sarabiliriz.İşte en iyi k araması için bir callbacks örneği:

In [19]:

class TopKTextGenerator(keras.callbacks.Callback):
    """top-k Kullanarak eğitilmiş bir modelden metin oluşturmak için bir callbacks"""

    def __init__(self, k):
        self.k = k

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = keras_nlp.utils.top_k_search(
            token_logits_fn,
            prompt_tokens,
            max_length=NUM_TOKENS_TO_GENERATE,
            k=self.k,
            from_logits=True,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search üretilen metin: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Geri aramayı göstermek için dummy train döngüsü.
model.fit(train_ds.take(1), verbose=2, epochs=2, callbacks=[text_generation_callback])

Epoch 1/2
Top-K search üretilen metin: 
b'[BOS] " i was not at present , " the captain replied . " if the men could not have had the slightest chance of obtaining , the captain would , as they did , if the boat had been affected by a slight shock . but , in the morning i should have taken up his position , and as i have no doubt that he was a prisoner , and i should have no'

1/1 - 10s - loss: 3.8801 - perplexity: 48.5693 - 10s/epoch - 10s/step
Epoch 2/2
Top-K search üretilen metin: 
b'[BOS] " yes , " answered the guide , but it is only possible that the guide \' s trail is the way in that direction is not so wide - awake . the road is a mile or so . i donned the track by track and passing along the trail of the path by a path leading down the path , and when i saw it was a short one , and i heard the noise that'

1/1 - 9s - loss: 3.8640 - perplexity: 47.7789 - 9s/epoch - 9s/step


## Özet

Özetlemek gerekirse, bu örnekte, bir kelime sözlüğünü eğitmek için KerasNLP katmanlarını kullanıyoruz.

Eğitim verilerini tokenize ettik, minyatür bir GPT modeli oluşturduk ve
metin oluşturma kitaplığını kullandık.

Transformers'ın nasıl çalıştığını anlamak veya Transformers'ı eğitmek hakkında daha fazla bilgi edinmek istiyorsanız,

- [Vaswani ve diğerleri, 2017](https://arxiv.org/abs/1706.03762)
- GPT-3 Raporu [Brown ve diğerleri, 2020](https://arxiv.org/abs/2005.14165)